In [1]:
import tensorflow as tf
import numpy as np
from sdm_ml.dataset import BBSDataset
from sklearn.preprocessing import StandardScaler
from block_diag import block_diagonal

In [2]:
csv_folder = '../../multi_species/bbs/dataset/csv_bird_data/'
dataset = BBSDataset(csv_folder)

In [3]:
training_set = dataset.get_training_set()

In [4]:
training_set.keys()

dict_keys(['covariates', 'outcomes'])

In [5]:
covariates = training_set['covariates']
outcomes = training_set['outcomes']

scaler = StandardScaler()

# Keep only the first 200 rows and 4 outcomes for this test
covariates = scaler.fit_transform(covariates.values[:200].astype(np.float32))
outcomes = outcomes.values[:200, :4]

In [6]:
n_features = covariates.shape[1]
n_outcomes = outcomes.shape[1]

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_features))
y = tf.placeholder(tf.int64, shape=(None, n_outcomes))

In [ ]:
# Next steps:
# 1. Implement the ARD RBF kernel in tensorflow.
# 2. Implement the site kernel in tensorflow (should have derivation).
# 3. Check that they look reasonable, maybe even on a super-small set.
# 4. Try to optimise with fixed hyperparameters.
# 5. Try to optimise the hyperparameters in an outer loop (Laplace approx.).
# 6. Compare speeds of sparse vs. non-sparse versions.